In [ ]:
%run ../code/init_mooc_nb.py

# General things
import warnings
from matplotlib import cm
from scipy import linalg as la
from functools import reduce
warnings.simplefilter("ignore", UserWarning)



# Defining systems
sigma0 = np.array([[1., 0.], [0., 1.]])
sigmax = np.array([[0., 1.], [1., 0.]])
sigmay = np.array([[0., -1j], [1j, 0.]])
sigmaz = np.array([[1., 0.], [0., -1.]])


def onsite(site, par):
    return ((2*par.t - par.mu) * np.kron(sigmaz, sigma0) + par.B * np.kron(sigma0, sigmaz) 
            + par.delta * np.kron(sigmax, sigma0))


def hopping(site1, site2, par):
    return -par.t * np.kron(sigmaz, sigma0) + 0.5 * 1j * par.alpha * np.kron(sigmaz, sigmax)


def nanowire_chain(L=10):
    lat = kwant.lattice.chain()
    sys = kwant.Builder()    
    
    sys[(lat(x) for x in range(L))] = onsite
    sys[kwant.HoppingKind((1,), lat)] = hopping
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1,)))
    lead[lat(0)] = onsite
    lead[kwant.HoppingKind((1,), lat)] = hopping
    
    sys.attach_lead(lead)
    sys.attach_lead(lead.reversed())
    
    return sys.finalized()


checkerboard = kwant.lattice.general([[2, 0], [1, 1]], [(0, 0), (1, 0)])
a, b = checkerboard.sublattices
T1, T2 = checkerboard.vec((1, 0)), checkerboard.vec((0, 1))


def get_infnite_2D_system():
    lat = kwant.lattice.chain()
    sys = kwant.Builder()
    
    def onsite(site, par):
        cell_hamiltonian = -par.t1 * np.array([[0, 1], [1, 0]])

        inter_cell_hoppings = {}
        inter_cell_hoppings[1, 0] = -par.t3 * np.array([[0, 1], [0, 0]])
        inter_cell_hoppings[0, 1] = -par.t4 * np.array([[0, 1], [0, 0]])
        inter_cell_hoppings[-1, 1] = -par.t2 * np.array([[0, 0], [1, 0]])

        result = np.array(cell_hamiltonian, dtype=complex)

        for key, val in list(inter_cell_hoppings.items()):
            vec = key[0] * T1 + key[1] * T2
            exp = np.exp(-1j * np.dot([par.kx, par.ky], vec))
            result += exp*val + exp.conj() * val.T.conj()
        return result
    
    sys[lat(0)] = onsite
    return sys.finalized()


def make_ribbon_lead(W):
    def lead_shape(pos):
        (x, y) = pos
        return (0 <= y < W and 0 <= x < W)


    lead = kwant.Builder(kwant.TranslationalSymmetry((1, 1)))
    lead[a.shape(lead_shape, (0,0))] = 0
    lead[b.shape(lead_shape, (1,0))] = 0
    

    lead[kwant.HoppingKind((0,0), b, a)] = lambda s1, s2, par: -par.t1
    lead[kwant.HoppingKind((-1,1), b, a)] = lambda s1, s2, par: -par.t2
    lead[kwant.HoppingKind((1,0), a, b)] = lambda s1, s2, par: -par.t3
    lead[kwant.HoppingKind((0,1), a, b)] = lambda s1, s2, par: -par.t4

    return lead.finalized()


# Helper functions
def get_evolution_operator(hamiltonians, T):
    n = float(len(hamiltonians))
    exps = [la.expm(-1j * h * T / n) for h in hamiltonians]
    return reduce(np.dot, exps)


def get_h_k(lead, par):
    bands = kwant.physics.Bands(lead, args=[par])
    h, t = bands.ham, bands.hop
    return lambda k: h + t * np.exp(-1j * k) + t.T.conj() * np.exp(1j * k)


#3D plots
def plot_2D(X,Y,Z, ax_in=None):
    if ax_in==None:
        fig = plt.figure(figsize=(7,5))
        ax = fig.add_subplot(111, projection='3d')
    else:
        ax = ax_in

    vmin = np.array(Z).min()
    vmax = np.array(Z).max()
    
    if len(np.shape(Z)) > 2:
        for z in Z:
            ax.plot_surface(X, Y, z, rstride=1, cstride=1, cmap=cm.RdBu_r, linewidth=0.1, vmin=vmin, vmax=vmax)
    else:
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.RdBu_r, linewidth=0.1, vmin=vmin, vmax=vmax)

    if ax_in==None:
        return fig, ax
    else:
        return ax 
    
    
def evaluate_on_grid(X, Y, func):
    """ X, Y should be in np.meshgrid form. It's enough for func to work on floats. """
    data = []
    for xx, yy in zip(X, Y):
        row = []
        for i,j in zip(xx, yy):
            row.append(func(i,j))
        data.append(row)
    data = np.array(data)
    data = [np.array(data[:,:,i]) for i in range(np.shape(data)[2])]
    return data

#Table of Contents
* [Introduction](#Introduction)
* [Periodically driven systems](#Periodically-driven-systems)
	* [Floquet theory](#Floquet-theory)
* [Driven Majorana wire](#Driven-Majorana-wire)
* [A Floquet Chern insulator](#A-Floquet-Chern-insulator)
* [Bulk-edge correspondence in driven systems](#Bulk-edge-correspondence-in-driven-systems)
* [Conclusions](#Conclusions)


**No content above this line is visible in edX**

# Introduction

Today's topic, Floquet topological insulators, is introduced by Mark Rudner from the Niels Bohr Institute at Copenhagen.

In [ ]:
MoocVideo("1peVp_IZ7Ts", src_location="11.1-intro")

# Periodically driven systems

The new generalization of the topology we will consider now is considering quantum evolution of systems with a time-dependent Hamiltonian. If you remember we've already encountered time dependence when we considered quantum pumps. However then we have assumed that the time evolution is very slow, and the system stays in the ground state at all times. But can we relax adiabaticity constraint? Can we find any analog of topology in systems driven so fast that the energy isn't conserved?

For the same reasons as before, we'll consider periodic driving

$$
H(t + T) = H(t).
$$

This is once again necessary because otherwise any system can be continuously deformed into any other, and there is no way to define a gap.

Before we get to topology, let's refresh our knowledge of time-dependent systems.

The Schrodinger equation gives us:

$$
i\frac{d \psi}{dt} = H(t) \psi
$$

It's linear, so we can write its solution as

$$
\psi(t_2) = U(t_2, t_1) \psi(t_1),
$$

with $U$ being a unitary *time evolution operator*. It solves the same Schrodinger equation as the wave function and it is equal to identity matrix at the initial time. It is commonly written as

$$
U(t_2, t_1) = \mathcal{T} \exp\,\left[-i\int_{t_1}^{t_2} H(t) dt\right]
$$

where $\mathcal{T}$ represents time-ordering (and not time-reversal symmetry). The time-ordering is just a short-hand notation for the need to solve the full differential equation, and it is necessary if Hamiltonians evaluated at different times in the integral do not commute.

The evolution operator safisfies a very simple multiplication rule:

$$
U(t_3, t_1) = U(t_3, t_2) U(t_2, t_1),
$$

which just says that time evolution from $t_1$ to $t_3$ is a product of time evolutions from $t_1$ to $t_2$ and then from $t_2$ to $t_3$. Of course an immediate consequence of this is the equality $U(t_2, t_1)^\dagger = U(t_1, t_2)$.

## Floquet theory

The central object for the study of driven systems is the evolution operator over one period of the drive:

$$
U(t + T, t) \equiv U.
$$

It is important because it allows us to find the wave functions that do not change if we wait for an arbitrary number of drive periods. These are the stationary states of a driven system, and they are given by the eigenvalues of the Floquet operator:

$$
U \psi = e^{i \alpha} \psi
$$

the stationary states are extremely similar to the eigenstates of a stationary Hamiltonian, with the only difference that they are only stationary if we look at fixed times $t + nT$. That's why the Floquet time evolution operator is also called stroboscopic time evolution operator.

We can very easily construct a Hermitian matrix from $U$, the **Floquet Hamiltonian**:

$$
H_\textrm{eff} = i T^{-1} \,\log U.
$$

Its eigenvalues $\varepsilon = \alpha / T$ are called quasi-energies, and they are always belonging to the interval $-\pi < \alpha \leq \pi$.

If the system is translationally invariant, we can study the effective band structure of $H_\textrm{eff}(\mathbf{k})$, find an energy in which the bulk Hamiltonian has no states, and study the topological properties of such a Hamiltonian: most of the things we already know still apply.

Of course, selecting a single quasi-energy as the Fermi level is arbitrary, since the equilibrium state of driven systems doesn't correspond to a Fermi distribution of filling factors, but at least it seems close enough for us to try applying topological ideas.

In [ ]:
question = ("But wait, we arbirarily chose the starting point $t$ in time for calculating the "
            "Floquet operator. What if we chose a different one?")

answers = ["The starting time is just an extra parameter of our system, and topology depends on it.",
           "It doesn't matter, the wave function evolution within one period "
           "can be neglected, since we are interested in many periods.",
           "There's only one correct starting point in time.",
           "It doesn't matter since the quasienergies are independent on the choice of the starting point."]

explanation = ("Choosing a different starting point applies a unitary transformation "
               "to the Floquet evolution operator, and so it keeps the quasienergies the same.")

MoocMultipleChoiceAssessment(question=question, answers=answers, correct_answer=3, explanation=explanation)

# Driven Majorana wire

Let us start from something we know very well: the superconducting Majorana nanowire model from week 2. This model has three important parameters which determine whether the wire is in the Majorana phase or not: the chemical potential $\mu$, the superconducting gap $\Delta$ and the magnetic field $B$. The topological phase with zero energy unpaired Majorana modes is realized for $B > \sqrt{\mu^2 + \Delta^2}$.

Let us now imagine that we can periodically drive some of these parameters. The easiest we can do is to take

$$
\mu = \left\{
\begin{matrix}
\mu_1 \quad \text{for } 0 < t < T/2 \\
\mu_2 \quad \text{for } T/2 < t < T
\end{matrix}\right.
$$

Then we don't need to take the integral, and we simply have

$$
U = \exp(i T H_2 / 2) \exp(i T H_1 / 2)
$$

with $H_1$ and $H_2$ the two nanowire Hamiltonians with chemical potential $\mu_1$ and $\mu_2$. A peculiar property of the driven systems is that as the period becomes large, the band structure 'folds': if the driving is very weak, and the original Hamiltonian had energy $E$, the Floquet Hamilotnian will have a much smaller quasienergy $(E\bmod 2\pi /T)$. This means that even whem $\mu_1$ and $\mu_2$ correspond to trivial systems, and just make the period large enough, we can still get nontrivial topology, take a look:

In [ ]:
def calculate_finite_spectrum(periods, hamiltonians):  
    energies = []
    for T in periods:
        U = get_evolution_operator(hamiltonians, T)
        
        phases = np.angle(la.eigvals(U))
        phases = np.sort(np.abs(phases))
        ev = np.sort([(-1)**n * val for n, val in enumerate(phases)])
        
        energies.append(ev)
    return np.array(energies).real


def calculate_bands(momenta, hamiltonians_k, T):
    energies = []
    for k in momenta:
        hamiltonians = [h_k(k) for h_k in hamiltonians_k]
        U = get_evolution_operator(hamiltonians, T)
        
        phases = np.angle(la.eigvals(U))
        phases = np.sort(np.abs(phases))
        ev = np.sort([(-1)**n * val for n, val in enumerate(phases)])
        
        energies.append(ev)
    return np.array(energies).real


J = 2.
par1 = SimpleNamespace(t=J/2, mu=-1*J, B=J, delta=2*J, alpha=J)
par2 = SimpleNamespace(t=J/2, mu=-3*J, B=J, delta=2*J, alpha=J)

sys = nanowire_chain(20)
H1 = sys.hamiltonian_submatrix(args=[par1])
H2 = sys.hamiltonian_submatrix(args=[par2])

h1_k = get_h_k(sys.leads[0], par1)
h2_k = get_h_k(sys.leads[0], par2)

periods = np.linspace(0.2/J, 1.6/J, 100)
momenta = np.linspace(-np.pi, np.pi)

energies = calculate_finite_spectrum(periods, [H1, H2])
spectrum = [calculate_bands(momenta, [h1_k, h2_k], T) for T in periods]


def plot(n):
    T = J * periods[n]
    
    fig, axes = plt.subplots(1,2, figsize=[9.5,4], tight_layout=True);
    ax1, ax2 = axes
    
    
    # Plotting data
    ax1.plot(J*periods, energies, 'k-')  
    ax1.plot([T,T],[-np.pi, np.pi], 'b--')
    ax2.plot(momenta, spectrum[n], 'k-')
    
    # Setting labels and titles
    ax1.set_xlabel(r'Driving period $(JT)$')
    ax1.set_ylabel(r'Quasi-energy $(ET)$')

    ax2.set_xlabel('$k$')
    ax2.set_ylabel('$E_kT$')
    
    ax1.set_title('finite system')
    ax2.set_title('Floquet bands')
    
    # Setting ticks
    vals = np.arange(0.2, 1.6, 0.4)
    ax1.set_xticks(vals)
    ax1.set_xticklabels(["${0}$".format(i) for i in vals])

    ax1.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
    ax1.set_yticklabels([r"$-\pi$", r"$-\frac{\pi}{2}$", r"$0$", r"$\frac{\pi}{2}$", r"$\pi$"])

    ax2.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
    ax2.set_xticklabels([r"$-\pi$", r"$-\frac{\pi}{2}$", r"$0$", r"$\frac{\pi}{2}$", r"$\pi$"])

    ax2.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
    ax2.set_yticklabels([r"$-\pi$", r"$-\frac{\pi}{2}$", r"$0$", r"$\frac{\pi}{2}$", r"$\pi$"])
    
    # Setting limits
    ax1.set_xlim(min(J*periods), max(J*periods))
    ax1.set_ylim(-np.pi*1.1, np.pi*1.1)
    
    ax2.set_xlim(-np.pi,np.pi);
    ax2.set_ylim(-np.pi*1.1, np.pi*1.1)

    return fig
    
StaticInteract(lambda n: plot(n), n = RangeWidget(0, len(periods)-5, 5, default=20))

On the left you see the Floquet spectrum of a finite system as a function of the driving period measured in the units of hopping strength, and on the right you see the Floquet dispersion in momentum space.

We now witness a cool phenomenon: just like in the undriven case, the particle-hole symmetry maps $E \rightarrow -E$, but now this means that not only $E = 0$ is special, but also $E = \pi$!

In other words, this means that there are two relevant gaps in the effective Floquet BdG Hamiltonian $H_\textrm{eff}$. Now, by using the same argument as the one which we used for the regular Majoranas, we learn that if we have an isolated Floquet state with a quasi-energy $\epsilon=0$ or $\epsilon=\pi$, it cannot be removed unless the gap surrounding it closes.

In other words:

> A Floquet superconductor has two types of Majorana bound states: the usual ones with quasienergy $E=0$, and the $\pi$-Majoranas that are as far from zero energy as possible.

So we learn two interesting features of driven systems revealed in the calculation shown above: the first is that the periodic driving can turn a trivial system into non-trivial system with topologically protected Floquet states. The second is that topology is richer than in the non-driven system: for instance, here the richness comes from the fact that the topologically protected states may occur at two different points in the spectrum.

Now try to answer the following question: what's the topological invariant of this system? How to tell when there is a $\pi$-Majorana, and when there's a regular Majorana being present? (We'll return to this question in the end of the lecture.)

# A Floquet Chern insulator

As the second example of a driven system that shows something that a regular system doesn't, let's consider the following toy model.

We take a square lattice with time-dependent nearest neighbor hopping $t$. Next, let's engineer a time-evolution of the hopping between sites such that during a period $T$ hoppings are turned on in an alternate fashion, as in the following figure:

![](figures/time_steps.png)

Each step lasts one quarter of a period.

Now let's tune the period such that the probability for an electron to hop along a hopping is one at the end of each quarter period [$t = (\pi / 2) / (T / 4)$]. Now over the complete period the trajectories of electrons will look like this:

![](figures/floquet_bulk.png)

Every electron makes a closed loop and ends up back at its origin. After every single period the system is back to its initial state. In other words, the Floquet operator $U=1$, and $H_\textrm{eff}=0$.

Let's check it (and also see what happens as we tune the driving period):

In [ ]:
def plot_dispersion_2D(T):
    def get_2D_h(par):
        sys = get_infnite_2D_system()

        def f(kx, ky):
            par.kx = kx
            par.ky = ky
            ham = sys.hamiltonian_submatrix(args=[par])
            return ham

        return f

    h1 = get_2D_h(SimpleNamespace(t1=1, t2=0, t3=0, t4=0))    
    h2 = get_2D_h(SimpleNamespace(t1=0, t2=1, t3=0, t4=0))    
    h3 = get_2D_h(SimpleNamespace(t1=0, t2=0, t3=1, t4=0))    
    h4 = get_2D_h(SimpleNamespace(t1=0, t2=0, t3=0, t4=1))

    hamiltonians_k = [h1, h2, h3, h4]

    def get_energies(kx, ky):
        hamiltonians = [h_k(kx, ky) for h_k in hamiltonians_k]
        U = get_evolution_operator(hamiltonians, T)
        ev = np.sort(np.angle(la.eigvals(U)))
        return ev

    K = np.linspace(-np.pi, np.pi, 50)
    mesh = np.meshgrid(K, K)
    energies = evaluate_on_grid(*mesh, func=get_energies)
    fig, ax = plot_2D(*mesh, Z=energies)

    ax.set_xlabel('$k_x$')
    ax.set_xticks([-np.pi, 0.0, np.pi])
    ax.set_xticklabels(['$-\pi$', '$0$', '$\pi$'])
    ax.set_ylabel('$k_y$')
    ax.set_yticks([-np.pi, 0.0, np.pi])
    ax.set_yticklabels(['$-\pi$', '$0$', '$\pi$'])
    ax.set_zlabel('$E$')
    ax.set_zticks([-3, 0.0, 3])
    ax.set_zticklabels(['$-3$', '$0$', '$3$'])
    
    ax.set_title(r'$T= %3.2f \pi$' % (T/np.pi))
    
    ax.view_init(8,20)
    ax.set_zlim3d(-4,4)
    
    return fig

N = 6
StaticInteract(lambda T: plot_dispersion_2D(2*np.pi*T / N), T=RangeWidget(N/2,3*N/2, default=N))

Now, there isn't a Hamiltonian which is more topologically trivial than the zero Hamiltonian. We may be tempted to conclude that our system is trivial and, by the bulk-boundary correspondence, has no edge states.

That's something we can also very easily verify by computing the dispersion of a finite size ribbon:

In [ ]:
def calculate_bands(momenta, hamiltonians_k, T):
    energies = []
    for k in momenta:
        hamiltonians = [h_k(k) for h_k in hamiltonians_k]
        U = get_evolution_operator(hamiltonians, T)
        energies.append(np.sort(np.angle(la.eigvals(U))))
    return np.array(energies).real


ribbon_lead = make_ribbon_lead(10)
h1_k = get_h_k(ribbon_lead, SimpleNamespace(t1=1, t2=0, t3=0, t4=0))
h2_k = get_h_k(ribbon_lead, SimpleNamespace(t1=0, t2=1, t3=0, t4=0)) 
h3_k = get_h_k(ribbon_lead, SimpleNamespace(t1=0, t2=0, t3=1, t4=0))    
h4_k = get_h_k(ribbon_lead, SimpleNamespace(t1=0, t2=0, t3=0, t4=1))

hamiltonians_k = [h1_k, h2_k, h3_k, h4_k]

periods = np.linspace(0, 4*np.pi, 11)
momenta = np.linspace(-np.pi, np.pi)
spectrum = [calculate_bands(momenta, hamiltonians_k, T) for T in periods]


def plot(n):
    T = periods[n]
    
    fig, ax = plt.subplots(1,1, figsize=[9.5,4], tight_layout=True);
    ax.set_color_cycle('k')
        
    ax.plot(momenta, spectrum[n])
    ax.set_title(r'spectrum: $T= %3.2f \pi$' % (T/np.pi))
    ax.set_xlim(-np.pi*1.05, np.pi*1.1)
    ax.set_ylim(-np.pi*1.05, np.pi*1.1)
    
    ax.set_xlabel('$k$')
    ax.set_ylabel('$E_kT$')
    
    ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
    ax.set_xticklabels([r"$-\pi$", r"$-\frac{\pi}{2}$", r"$0$", r"$\frac{\pi}{2}$", r"$\pi$"])

    ax.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
    ax.set_yticklabels([r"$-\pi$", r"$-\frac{\pi}{2}$", r"$0$", r"$\frac{\pi}{2}$", r"$\pi$"])
    

    
    return fig

StaticInteract(lambda n: plot(n), n = RangeWidget(0, len(periods)-1, default=int(len(periods)/2)))

We see something very different from our expectations. All the bulk states are indeed at $E=0$, but there are two branches of dispersion, that are clearly propagating. These can only belong to the edges, and since the two edges look identical, these two modes have to belong to the opposite edges. We seem to conclude that even though the bulk Hamiltonian is trivial, the edges carry chiral edge states, as if there was a finite Chern number.

When the driving period is tuned to ensure the absence of bulk dispersion, we can also understand why the edge states appear. If we select a state that starts on the edge, and follow it for one period, we'll find that there are modes that just never leave the edge, since one of the hoppings in the vertical direction is absent.

![](figures/trajectories.png)

So what is happening with the bulk-edge correspondence?

In [ ]:
question = ("How can you change the chirality of the edge states in the figure above?")

answers = ["By changing the driving period.",
           "By reversing the driving protocol sequence.",
           "By changing the sign of the nearest neighbor hopping.",
           "By making the electrons start from the black sublattice."]

explanation = ("Reversing the driving protocol is the same as applying time-reversal symmetry, "
               "so it will reverse the direction of the chiral edge modes")

MoocMultipleChoiceAssessment(question=question, answers=answers, correct_answer=1, explanation=explanation)

# Bulk-edge correspondence in driven systems

The two examples we've studied reveal an imporant feature of topological Floquet insulators. It seems that knowing the bulk Floquet Hamiltonian is sufficient to calculate the topological invariant by just applying the known expression to the Floquet Hamiltonian, but that's not enough.

In rough terms, the reason for this insufficiency is due to Floquet topological insulators missing a topologically trivial state which can be taken as a reference. With any regular 2D Hamiltonian, we know that if we take $E \rightarrow -\infty$, we will get a trivial system with the Chern number zero. In a Floquet system, the only thing lowering the energy tells is is that the Chern number is periodic in quasienergy like any other observable property.

What do we need to know to derive the full topological invariant from the bulk properties? The answer is that we need the complete evolution operator for any moment in time, or in other words the full dependence $H(t)$. The actual calculation of the topological invariant is technically involved, and falls beyond what we can cover in this course. Moreover, to the best of our knowledge the full classification of Floquet topological insulators is not yet accomplished.

# Conclusions

In [ ]:
MoocVideo("DbyqIczcR9c", src_location="11.1-summary")

Questions about what you just learned? Ask them below!

In [ ]:
MoocDiscussion("Questions", "Floquet")